In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [3]:
data

{'data': [array([[ 0.02195837,  0.15468682, -0.14086571, -0.03870261, -0.02876111,
           0.23291191, -0.01158469, -0.03941091, -0.01008287,  0.01623408,
           0.15685216,  0.06436369,  0.1210824 , -0.11064344, -0.10362137,
          -0.04955597, -0.06856476,  0.05500543, -0.13561653,  0.11160272,
           0.05425845, -0.01279023, -0.00130818,  0.11921053, -0.08102003,
          -0.01894449,  0.00644443, -0.08794577,  0.04482058,  0.06944231,
           0.03444667,  0.01894323, -0.03860816,  0.11403698, -0.06041174,
           0.08976212, -0.01103042,  0.03768162, -0.05673286,  0.0765649 ,
           0.02396066, -0.1403357 , -0.05571873,  0.11809073, -0.12817422,
          -0.1424281 ,  0.1421219 ,  0.07524506, -0.14461058,  0.11972223,
           0.03312606,  0.03032682, -0.14932239,  0.03276768,  0.03689465,
           0.08104338, -0.09758465,  0.11989936, -0.01355501, -0.08075338,
          -0.16726407, -0.00525313,  0.28831947, -0.19264154,  0.02504505,
          -0.0919

In [4]:
X = np.array(data['data']) # indendepent variable
y = np.array(data['label']) # dependent variable

In [5]:
X.shape , y.shape

((1654, 1, 128), (1654,))

In [6]:
X = X.reshape(-1,128)
X.shape

(1654, 128)

In [7]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [8]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)

In [9]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1323, 128), (331, 128), (1323,), (331,))

In [10]:
# 2. train machine learning

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [12]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training logistic regression

LogisticRegression()

In [13]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test = f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 Score Train = %0.2f'%f1_score_train)
    print('F1 Score Test = %0.2f'%f1_score_test)

In [14]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.73
Accuracy Test = 0.67
F1 Score Train = 0.73
F1 Score Test = 0.66


In [15]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [16]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.84
Accuracy Test = 0.69
F1 Score Train = 0.84
F1 Score Test = 0.68


In [17]:
model_rf = RandomForestClassifier(n_estimators=10,)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [18]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00
Accuracy Test = 0.57
F1 Score Train = 1.00
F1 Score Test = 0.56


In [19]:
# 3. voting classifiers

In [20]:
model_voting = VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [21]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft', weights=[2, 3, 1])

In [22]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.88
Accuracy Test = 0.69
F1 Score Train = 0.88
F1 Score Test = 0.69


In [23]:
# parameter tuning

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft','hard']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [26]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.4s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3

[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.7s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=3, rf__n_estimators=20, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.4s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.7s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.1, voting=hard; total time=   0.4s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.4s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=5, svm__C=5, svm__gam

[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.4s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.3, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.4s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=10, svm__C=5

[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=5, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=soft; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=5, rf__n_estimators=20, svm__C=7

[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.4s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=soft; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.4s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=5, svm__C=7, svm__gam

[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.3, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=soft; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.4s
[CV] END rf__max_depth=7, rf__n_estimators=10, svm__C=

[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=7, svm__gamma=0.5, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=soft; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.5s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.1, voting=hard; total time=   0.6s
[CV] END rf__max_depth=7, rf__n_estimators=20, svm__C=10, svm__gamma=0.3, voting=soft; total time=   0.7s
[CV] END rf__max_depth=7, rf__n_estimators=20, s

GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier())],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [27]:
model_grid.best_estimator_

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(C=7, gamma=0.3, probability=True)),
                             ('rf',
                              RandomForestClassifier(max_depth=5,
                                                     n_estimators=10))],
                 voting='soft', weights=[2, 3, 1])

In [28]:
model_best_estimator = model_grid.best_estimator_

In [29]:
model_grid.best_score_

0.708994708994709

In [30]:
# save model as pickle file

In [31]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))